- https://mjdeeplearning.tistory.com/97
- https://hayunjong83.tistory.com/47

### ImageFolder
torchvision.datasets.ImageFolder를 호출하면 갖춰진 데이터 폴더 구조로부터 PyTorch dataset을 만들어준다.  
> 내부적으로 샘플에 대한 인덱싱을 제공하고, 문자열로 된 클래스명들을 0번 클래스, 1번 클래스와 같이 정수 인덱스로 바꿔주는 작업이 이뤄지지만, 이런 복잡한 내부 작업을 숨기고 간단한 메소드 호출만으로 작업이 이뤄지게 하는 것이 라이브러리의 힘이다.  
  
ImageFolder 메소드는 ants/bees가 속한 train 폴더로부터 train dataset을, ants/bees가 속한 val 폴더로부터 validation dataset을 자동으로 만들어줄 것이다.  
PyTorch dataset을 만들면서 torchvision 라이브러리를 쓰는 방식의 장점은 이미지 변형(Image transformation)을 손쉽게 할 수 있다는 점이다. 
### Transforms
torchvision.transforms을 이용해서 변형 방식을 구성한 다음에 dataset을 만들 때 넘겨주는 것으로 충분하다.  
- transforms.Compose: 여러 변형 방식을 한꺼번에 묶어준다.
- transforms.RandomResizedCrop: 원본 이미지를 지정된 스케일 범위와 지정된 비율로 무작위로 자른 후에, 지정된 크기로 리사이즈 한다. 
- transforms.Resize: 입력 이미지를 주어진 크기로 리사이즈한다.
- transforms.CenterCrop: 입력 이미지의 중앙에서 시작하여 주어진 크기만큼 크롭한다. 참고로 입력 이미지 크기가 출력하려는 크기보다 작으면 부족한 부분의 픽셀값들이 0으로 패딩, 즉 채워진 다음에 크롭된다.
- transforms.HorizontalFlip: 주어진 확률로 이미지를 수평반전 시킨다. 여기서는 확률값을 지정하지 않았으므로 디폴트 값인 0.5의 확률로 이미지들이 수평반전 된다. 즉, 훈련 이미지 중 절반은 그대로지만, 절반은 뒤집히게 될 것이다.
- transforms.ToTensor: 사전훈련된 모델 사용과 효율적 연산을 위해서 torch.FloatTensor 배열로 바꿔줘야 한다. 
> 이 때, 픽셀값의 범위가 [0.0, 1.0] 사이가 되도록 바꿔줘야하며 차원의 순서를 바꿔서 (채널수, 높이, 너비)가 되게 해야한다. 이 작업을 수행해주는 메소드이다.
- transforms.Normalize: transfer learning에서 사용하는 pretrain model들은 대게 ImageNet 데이터셋에서 사전훈련되어있다. 이 모델을 사용하기 위해서 ImageNet 데이터의 각 채널별 평균과 표준편차를 이용해 정규화해준다. 
> OpenCV를 사용해 이미지를 읽어온다면 RGB이미지가 아닌 BGR이미지이므로, 채널 순서를 염두에 둬야 할 것이다. 

In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F 
import torch.optim as optim 

import torchvision 
from torchvision import datasets, models, transforms 
from torch.utils.data import DataLoader

import os

In [6]:
data_transform = {
  'train': transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
  ]),
  'val': transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
  ])
}

data_dir = '../data/hymenoptera_data'
image_datasets = {
  x: datasets.ImageFolder(os.path.join(data_dir, x), data_transform[x]) for x in ['train', 'val']
}

dataloaders = {
  x: DataLoader(
    image_datasets[x], batch_size=4, shuffle=True, num_workers=4
  ) for x in ['train', 'val']
}

dataset_sizes = {
  x: len(image_datasets[x]) for x in ['train', 'val']
}
class_names = image_datasets['train'].classes

In [7]:
class_names

['ants', 'bees']

In [8]:
dataset_sizes

{'train': 244, 'val': 153}